In [1]:
%pip install -e ../
%pip show iesta


Obtaining file:///home/elba_ro/repos/github/conf22-style-transfer
  Attempting uninstall: iesta
    Found existing installation: iesta 1.2
    Uninstalling iesta-1.2:
      Successfully uninstalled iesta-1.2
  Running setup.py develop for iesta
Note: you may need to restart the kernel to use updated packages.
Name: iesta
Version: 1.2
Summary: Python package for Conf 2023 for INEFF-IFFE Style Transfer
Home-page: UNKNOWN
Author: Roxanne El Baff
Author-email: roxanne.elbaff@dlr.de
License: MIT
Location: /home/elba_ro/repos/github/conf22-style-transfer
Requires: empath, markupsafe, nlpaf, seaborn, spacy, tomli, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:

import iesta.machine_learning.dataloader
#from .autonotebook import tqdm as notebook_tqdm
import iesta.stats.significance
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(3,2)}, 
    #style="white" # nicer layout
)


/home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
feature_dfs = iesta.machine_learning.dataloader.load_features_df()
feature_dfs[1]

/home/elba_ro/repos/github/conf22-style-transfer/iesta/../data/splitted_conservative_debate_arguments_effect_test0.3_random2.parquet


iesta        INFO     File already created. Loading file...


/home/elba_ro/repos/github/conf22-style-transfer/iesta/../data/splitted_liberal_debate_arguments_effect_test0.3_random2.parquet


iesta        INFO     File already created. Loading file...


316: ../data/extracted_features//conservative_style-features_1000/conservative_batch1000_72_style-features.parquet
16: ../data/extracted_features//conservative_transformer-features_100/conservative_batch100_714_transformer-features.parquet
792: ../data/extracted_features//liberal_style-features_1000/liberal_batch1000_47_style-features.parquet
92: ../data/extracted_features//liberal_transformer-features_100/liberal_batch100_468_transformer-features.parquet


{'conservative':                                                     id  \
 97          0.999..-repeating-is-equal-to-1/1/|The_Tom   
 98          0.999..-repeating-is-equal-to-1/1/|The_Tom   
 99          0.999..-repeating-is-equal-to-1/1/|The_Tom   
 114           0.999999999....-is-equal-to-1/1/|The_Tom   
 115           0.999999999....-is-equal-to-1/1/|The_Tom   
 ...                                                ...   
 100002  the-Rothschilds-have-enough-control/1/|thefact   
 100003  the-Rothschilds-have-enough-control/1/|thefact   
 100004  the-Rothschilds-have-enough-control/1/|thefact   
 100005  the-Rothschilds-have-enough-control/1/|thefact   
 100535           the-wii-is-a-good-console/1/|sushovan   
 
                                      debate_id    p_name           effects  \
 97          0.999..-repeating-is-equal-to-1/1/   The_Tom         effective   
 98          0.999..-repeating-is-equal-to-1/1/   The_Tom         effective   
 99          0.999..-repeating-is-equ

In [3]:
significance = iesta.stats.significance.run_all_significance_test(feature_dfs[1])

Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}
Running {ideology} {iv} {excluded_effect}


In [4]:
for k in significance.keys():
    print(k)


def _get_key(ideology, diregard_okay=False, undersampled=False, binary_effect=False):
    _okay = "_Okay" if diregard_okay else ""
    _undersampled = "_undersampled" if undersampled else ""
    _binary = "_binary" if binary_effect else ""
    _key = f"{ideology}{_binary}_effect{_undersampled}{_okay}_all_features"
    return _key

def _get_significance(ideology, diregard_okay=False, undersampled=False, binary_effect=False):
    _key = _get_key(ideology, diregard_okay, undersampled, binary_effect)
    _df = significance[_key][0].reset_index().copy()
    _df = _df[(~_df["feature"].str.endswith("_count")) & (~_df["feature"].str.endswith("_ratio"))]
    if 'Unnamed: 0'  in _df.columns.tolist():
        del _df['Unnamed: 0']
    return _df

conservative_effect_undersampled_all_features
conservative_effect_all_features
conservative_effect_undersampled_okay_all_features
conservative_effect_okay_all_features
conservative_binary_effect_undersampled_all_features
conservative_binary_effect_all_features
conservative_binary_effect_undersampled_okay_all_features
conservative_binary_effect_okay_all_features
liberal_effect_undersampled_all_features
liberal_effect_all_features
liberal_effect_undersampled_okay_all_features
liberal_effect_okay_all_features
liberal_binary_effect_undersampled_all_features
liberal_binary_effect_all_features
liberal_binary_effect_undersampled_okay_all_features
liberal_binary_effect_okay_all_features


In [5]:
# LIBERAL BINARY - With NO OKAY

liberal_binary_nookay = _get_significance("liberal", diregard_okay=True, undersampled=False, binary_effect=True)
liberal_binary_nookay_undersampled = _get_significance("liberal", diregard_okay=True, undersampled=True, binary_effect=True)
liberal_binary_nookay, liberal_binary_nookay_undersampled

KeyError: 'liberal_binary_effect_Okay_all_features'

In [ ]:
cons_binary_nookay = _get_significance("conservative", diregard_okay=True, undersampled=False, binary_effect=True)
cons_binary_nookay_undersampled = _get_significance("conservative", diregard_okay=True, undersampled=True, binary_effect=True)
cons_binary_nookay, cons_binary_nookay_undersampled

In [ ]:
liberal_alleffect_nookay = _get_significance("liberal", diregard_okay=True, undersampled=False, binary_effect=False)
liberal_alleffect_nookay_undersampled = _get_significance("liberal", diregard_okay=True, undersampled=True, binary_effect=False)
liberal_alleffect_nookay
# liberal_alleffect_nookay_undersampled HAS 1 RESULT

In [ ]:
conservative_alleffect_nookay = _get_significance("conservative", diregard_okay=True, undersampled=False, binary_effect=False)
conservative_alleffect_nookay_undersampled = _get_significance("conservative", diregard_okay=True, undersampled=True, binary_effect=False)
conservative_alleffect_nookay
# liberal_alleffect_nookay_undersampled HAS 1 RESULT

In [ ]:
_get_key("liberal", diregard_okay=True, undersampled=False, binary_effect=False)

In [ ]:
features_df  = feature_dfs[1]["liberal"]

features_df["effect"].unique()

In [ ]:

features_df[~features_df["effect"].isin(["okay"])].copy()["effect"].unique()
